In [ ]:
import os

In [ ]:
os.chdir('../')
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PredictionConfig:
    path_of_model: Path
    pre_trained_face_detector: Path   

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])  

    def get_prediction_config(self) -> PredictionConfig:
        prediction_config = PredictionConfig(
            path_of_model= self.config.training.trained_model_path,           
            pre_trained_face_detector = 'haarcascade_frontalface_default.xml'
            )
        
        return prediction_config

In [ ]:
import sys
from cnnClassifier.exception import CustomException
import numpy as np
import cv2
from cnnClassifier.utils import load_object

In [ ]:
class Prediction:
    def __init__(self, filename: Path, config: PredictionConfig):
        self.config = config
        self.filename = filename

    def predict(self):
        emotion_dict = {0: "Neutral", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Sad", 5: "Surprised", 6: "Neutral"}
        
        model = load_object(self.config.path_of_model, h5=True)
        facecasc = cv2.CascadeClassifier(self.config.pre_trained_face_detector)
        image = cv2.imread(self.filename)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = facecasc.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=10)
        print("No of faces : ",len(faces))
        i =1
        for (x, y, w, h) in faces:
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            # If the input of trained model has one chanels
            #roi_gray = gray[y:y + h, x:x + w] 
            # If the input of trained model has three chanels         
            roi_color = image[y:y+h, x:x+w]               
            # Croping
            #cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_color, (48, 48)), -1), 0)
            prediction = model.predict(cropped_img)

            maxindex = int(np.argmax(prediction))            
            cv2.putText(image, emotion_dict[maxindex], (x+10, y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        file_name = os.path.basename(self.filename )
        dir_name = os.path.dirname(self.filename) 
        cv2.imwrite(os.path.join(dir_name, 'pred_'+file_name), image)

In [ ]:
try:
    config = configurationManeger()
    pred_config = config.get_prediction_config()
    prediction = Prediction('images/img2.png', pred_config)
    prediction.predict()
    
except Exception as e:
    raise CustomException(e, sys)